In [6]:
import cv2
import csv
import numpy as np
from os import listdir, makedirs, path
from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib import pylab
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
%matplotlib inline
pylab.rcParams['figure.figsize'] = (100, 50)

In [7]:
trf = 'Scan'  # subfolder scans
root_folder = Path('/home/simone/development/python_università/Mario/RAW DATA')
ext = '.tif'  # source file extention
ext_save = '.tif'  # save file extention

In [8]:
subdirs = [x.name for x in root_folder.iterdir() if x.is_dir()]

In [12]:
method = widgets.Dropdown(
    options=['BINARY', 'BINARY+OTSU', 'ADAPTIVE_MEAN', 'ADAPTIVE_GAUSSIAN'],
    value='BINARY',
    description='Method',
    disabled=False)
dir_selection = widgets.Dropdown(
    options=subdirs,
    value=subdirs[0],
    description='Folder',
    disabled=False
    )
file_selection = widgets.Dropdown(
    options=[each for each in listdir(root_folder / dir_selection.value) if each.endswith(ext)],
    description='Scan',
    disabled=False)
def update_file_list(folder):
    new_folder = root_folder / str(folder['new'])
    file_selection.options = [each for each in listdir(new_folder) if each.endswith(ext)]
dir_selection.observe(update_file_list, names='value')  #update file list on folder change giving the new value to update_file_list
cut_left = widgets.IntSlider(value = 10,
                             min = 0,
                             max = 50,
                             step = 1,
                             description = 'Cut_left',
                             continuous_update = False)
cut_right = widgets.IntSlider(value = 90,
                              min = 50,
                              max = 100,
                              step = 1,
                              description = 'Cut_right',
                              continuous_update = False)
cut_top = widgets.IntSlider(value = 5,
                            min = 0,
                            max = 50,
                            step = 1,
                            description = 'Cut_top',
                            continuous_update = False)
cut_bottom = widgets.IntSlider(value = 95,
                               min = 50,
                               max = 100,
                               step = 1,
                               description = 'Cut_bottom',
                               continuous_update = False)
threshold = widgets.IntSlider(value = 100,
                              min = 0,
                              max = 255,
                              step = 1,
                              description = 'Global thr',
                              continuous_update = False)
denoise = widgets.IntSlider(value = 3,
                            min = 0,
                            max = 20,
                            step = 1,
                            description = 'Denoise',
                            continuous_update = False)
min_area = widgets.IntSlider(value = 8000,
                            min = 0,
                            max = 30000,
                            step = 1000,
                            description = 'Min Area',
                            continuous_update = False)
max_area = widgets.IntSlider(value = 800000,
                            min = 600000,
                            max = 1000000,
                            step = 1000,
                            description = 'Max Area',
                            continuous_update = False)
blocksize = widgets.IntText(value = 101,
                            min = 1,
                            max = 201,
                            step = 2,
                            description = 'Blocksize',
                            continuous_update = False)

widget_conf = {
     'dir_selection': dir_selection,
     'file_selection': file_selection,
     'method': method,
     'min_area': min_area,
     'max_area': max_area,
     'cut_left': cut_left,
     'cut_right': cut_right,
     'cut_top': cut_top,
     'cut_bottom': cut_bottom,
     'threshold': threshold,
     'denoise': denoise,
     'blocksize': blocksize,
     'plot': True,
     'all_samples': True
}

In [13]:
def segmentation_func(dir_selection, file_selection, cut_left, cut_right, cut_top, cut_bottom, method, min_area, max_area, threshold, denoise, blocksize, plot, all_samples):
    print(f"processing: {locals()}")
    # must split this function in sub function
    fullname_fg = root_folder / dir_selection / file_selection #join paths using pathlib to abstract OS differences
    img = cv2.imread(str(fullname_fg))
    img_cut = img.copy()
    h, w = img_cut.shape[:2]
    w_left = int(w*cut_left/100)
    cv2.line(img_cut,(w_left, 0),(w_left, h),(0,255,0), int(w*0.01))
    w_right = int(w*cut_right/100)
    cv2.line(img_cut,(w_right, 0),(w_right, h),(255,255,0), int(w*0.01))
    h_top = int(h*cut_top/100)
    cv2.line(img_cut,(0, h_top),(w, h_top),(0,0,255), int(w*0.01))
    h_bottom = int(h*cut_bottom/100)
    cv2.line(img_cut,(0, h_bottom),(w, h_bottom),(255,0,255), int(w*0.01))
    img = img[h_top:h_bottom, w_left:w_right,:]
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    if method == 'BINARY':
        ret, th1 = cv2.threshold(gray,threshold,255,cv2.THRESH_BINARY)
    if method == 'BINARY+OTSU':
        ret, th1 = cv2.threshold(gray,threshold,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    if method == 'ADAPTIVE_MEAN':
        th1 = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, blocksize, 0)
    if method == 'ADAPTIVE_GAUSSIAN':
        gray = cv2.GaussianBlur(gray,(5,5),0)
        th1 = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, blocksize, 0)
    th1_floodfill = th1.copy()
    hf, wf = th1.shape[:2]
    mask = np.zeros((hf + 2, wf + 2), np.uint8)
    cv2.floodFill(th1_floodfill, mask, (0, 0), 255)
    th1_floodfill_inv = cv2.bitwise_not(th1_floodfill)
    th1 = th1 | th1_floodfill_inv
    kernel = np.ones((denoise,denoise),np.uint8)
    th1 = cv2.morphologyEx(th1,cv2.MORPH_OPEN,kernel, iterations = 2)
    n_samples = 0
    if plot == True:
        
        _, contours, hi = cv2.findContours(th1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #get countours
        #writer countors to image
        img_contours = img.copy()
        
        
        #img_contours = img.copy()
        #cropped_images = []
        for i, cnt in enumerate(contours):
            #X, Y, w, h = cv2.boundingRect(cnt)
            cv2.drawContours(img_contours, contours, i, (0, 255, 0), 3)
            #section = img[Y:Y + h, X:X + w]
            #cropped_images.append(section)
            area = cv2.contourArea(cnt)
            if (area < min_area or area > max_area):
                continue
            n_samples += 1
              
        images = [img_cut[:,:,::-1], th1, img_contours[:,:,::-1]]
        colors = [None, 'gray', None]
        for i in range(3):
            plt.subplot(1,3,i+1),plt.imshow(images[i], colors[i])
        plt.show()    
            
        # plot images
        #plot_rows = (len(cropped_images) // 3) + 4 # to ensure enogh space for cropped images plus orginal images
        # plot_cols = 3
        #for i in range(2):
            #plt.subplot(plot_rows, 2, i+1),plt.imshow(images[i], cols[i])
        # contours   
        #plt.subplot(plot_rows, 1 , 3)
        #plt.imshow(img_contours)
        # plot cropped images
        #for i, img in enumerate(cropped_images):
            #plt.subplot((len(cropped_images) // 3) +1 ,3 ,i + 4) # 1 + 3(first row)
            #plt.imshow(img)     
                
                   
        print("Detected samples:", str(n_samples))
    else:
        return th1, img

In [14]:

interact(segmentation_func, **widget_conf) # using widget_conf paramenter unpacking to simply this cell

interactive(children=(Dropdown(description='Folder', options=('Cetriolo', 'Peperoncino', 'Patata', 'Pomodoro -…

<function __main__.segmentation_func(dir_selection, file_selection, cut_left, cut_right, cut_top, cut_bottom, method, min_area, max_area, threshold, denoise, blocksize, plot, all_samples)>

!! questa cella è segnata come markdown e non codice!
for fname in flist:
    th1, img = segmentation_func(fname,
                            cut_left.value,
                            cut_right.value,
                            cut_top.value,
                            cut_bottom.value,
                            method.value,
                            threshold.value,
                            denoise.value,
                            blocksize.value,
                            plot = False)
    _, contours, hi = cv2.findContours(th1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    img_contours = img.copy()
    for i, cnt in enumerate(contours):
        X, Y, w, h = cv2.boundingRect(cnt)
        cv2.drawContours(img_contours, contours, i, (0, 255, 0), 3)
        section = img[Y:Y + h, X:X + w]
        path_name = "./Results/Sample_" + str(fname[:-len(ext)])
        if not path.exists(path_name):
            makedirs(path_name)
        cv2.imwrite(path_name + "/" + str(fname[:-len(ext)]) + "_crop_" + str(i) + ext_save, section)
    cv2.imwrite("./Results/" + str(fname[:-len(ext)]) + '_contours' + ext_save, img_contours)
    cv2.imwrite("./Results/" + str(fname[:-len(ext)]) + '_mask' + ext_save, th1)